### construire et dessiner l'évolution de la T°C en FRANCE depuis 1800

> source: [ici](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data?select=GlobalLandTemperaturesByCountry.csv)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from time import time
rng = np.random.default_rng(seed=int(time()))
pd.__version__

URL="https://raw.githubusercontent.com/bandeDeCodeurs/dataviz_temperatures_france/main/GlobalLandTemperaturesByCountry.csv"
FINAL_URL="https://fr.wikipedia.org/wiki/Climat_de_la_France"

In [ ]:
# téléchargement + écriture en local
pd.read_csv(
    URL,
).to_csv(
    "big_mean_global_temperature_1748_2013.csv",
    encoding="utf8",
    index=False
)

In [ ]:
temp_df = pd.read_csv(
    "big_mean_global_temperature_1748_2013.csv",
    encoding="utf8",
    index_col="dt",
    parse_dates=True
)
# compléter les données d'une autre source
## read_html : trouver les <table> HTML dans une liste, chaque élément est converti en dataframe
# attention aux formats en terme de décimaux et milliers
wiki_df = pd.read_html(FINAL_URL, thousands=".",  decimal=",")[2]
temp_df
wiki_df


In [ ]:
# temp_df["Country"].unique()
after_1800 = temp_df.index >= datetime.strptime("1800-01-01", "%Y-%m-%d")
temp_df = temp_df.loc[(after_1800) & (temp_df["Country"] == "France (Europe)")]
temp_df.drop(columns=["AverageTemperatureUncertainty", "Country"], inplace=True)
temp_df

In [ ]:
wiki_df = wiki_df.iloc[0:10, 0:2]
# wiki_df.drop(labels=wiki_df.index[10:]).drop(columns=wiki_df.columns[2:])

## remplacement vectorisé, utilisation de groupe de capture
# en oublie pas que le groupe n°0 c'est la regex elle même donc (\d{4}) es t le groupe n°1
# et on peut utiliser les références arrières en injectant le contenu d'un groupe ici \\1
wiki_df["Année"] = wiki_df["Année"].str.replace(pat="(\d{4})\[[a-z]\]", repl="\\1", regex=True)
# conversion en date
wiki_df["Année"] = pd.to_datetime(wiki_df["Année"], format="%Y")
wiki_df.set_index("Année", inplace=True)
wiki_df.sort_index(ascending=True, inplace=True)

wiki_df

In [ ]:
# pb T° par mois VS T° par année : version artisanale
def get_temp_year(df: pd.DataFrame, ind):
    year = df.index[ind].year
    tmp_df = df[ind:ind+12].reset_index()
    # shift: décaler les données par rapport à l'index vers le haut < 0 et le bas > 0
    tmp_df["dt"] = tmp_df["dt"].diff().shift(-1)
    tmp_df["dt"] = tmp_df["dt"].dt.days.fillna(31.).astype("i2")
    t = round(tmp_df.apply(lambda r: r["dt"]*r["AverageTemperature"], axis=1).sum()/365, 2)
    return pd.DataFrame(data=[[year, t]])

container = []
for i in range(0, 212 * 12, 12):
    container.append(get_temp_year(temp_df, i))
pd.concat(container, axis=0)

In [ ]:
# pb T° par mois VS T° par année : version magique
temp_df = temp_df.resample("Y", kind="period").mean().rename(columns={"AverageTemperature": "Moyenne annuelle"})
wiki_df.index = wiki_df.index.strftime("%Y")

In [ ]:
complete_df = pd.concat([temp_df, wiki_df], axis=0)
del temp_df
del wiki_df


In [ ]:
complete_df